In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Ranking FFT frequencies based on absolute value (sqrt(a^2+b^2))

phi = np.load("Phi_201snap.npy")
df = pd.DataFrame()

# Define values for k
k_values = [1]

for k in k_values:

    # Compute 1D FFTs (one along r and one along z)
    for ax in range(2):
        max1D, min1D = -1000, 1000
        sum_of_squares1D = 0
        
        for i in range(201):
            data = phi[:,:,i]
            fft = np.fft.fft(data, axis=ax)

            # Sort array by row by absolute value in descending order
            fft_sorted = np.flip(np.sort(np.abs(fft), axis=ax), axis=ax)

            # Take the top k frequencies from each row/col of fft_sorted by defining threshold & mask smaller values
            if ax == 0: # if FFT is along r 
                threshold = fft_sorted[k, :]
            else:       # if FFT is along z
                threshold = fft_sorted[:, k].reshape(101, 1)
        
            mask = np.abs(fft)>threshold
            masked = fft * mask
            newdata = np.fft.ifft(masked).real

            # Add squared differences to total sum, update min/max values for later
            diff1D = data - newdata
            sum_of_squares1D += np.sum(diff1D**2)
            max1D = max(max1D, np.max(diff1D))
            min1D = min(min1D, np.min(diff1D))

        # Compute RMSE's and CR's, add to dataframe
        range1D = max1D - min1D
        rms1D = np.sqrt(sum_of_squares1D / (101*227*201))
        rmse1D = rms1D / (100*range1D)
        cr1D = data.shape[ax] / k
        cr_string1D = "CR1D_k=" + str(k) + "_ax=" + str(ax)
        rmse_string1D = "RMSE1D_k=" + str(k) + "_ax=" + str(ax)
        df.at[0, cr_string1D] = cr1D
        df.at[0, rmse_string1D] = rmse1D

    # Compute 2D FFTs
    max2D, min2D = -1000, 1000
    sum_of_squares2D = 0
    
    for i in range(201):
        data = phi[:,:,i]
        fft2 = np.fft.fft2(data)
            
        # Sort fft by absolute value in descending order
        fft2_sorted = np.sort(np.abs(fft2.reshape(-1)))[::-1]
    
        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold2 = fft2_sorted[k]
        mask2 = np.abs(fft2)>threshold2
        masked2 = fft2 * mask2
        newdata2 = np.fft.ifft2(masked2).real
        
        # Add squared differences to total sum, update min/max values for later
        diff2D = data - newdata2
        sum_of_squares2D += np.sum(diff2D**2)
        max2D = max(max2D, np.max(diff2D))
        min2D = min(min2D, np.min(diff2D))

    range2D = max2D - min2D
    rms2D = np.sqrt(sum_of_squares2D / (101*227*201))
    rmse2D = rms2D / (100*range2D)
    cr2D = (101*227) / k
    cr_string2D = "CR2D_k=" + str(k)
    rmse_string2D = "RMSE2D_k=" + str(k)
    df.at[0, cr_string2D] = cr2D
    df.at[0, rmse_string2D] = rmse2D
        
df.to_pickle("analysis2.4.pkl")
df

,CR1D_k=1_ax=0,RMSE1D_k=1_ax=0,CR1D_k=1_ax=1,RMSE1D_k=1_ax=1,CR2D_k=1,RMSE2D_k=1
0,101.0,0.000027,227.0,0.00022,22927.0,0.000256


In [68]:
# Testing
# test = np.random.rand(3, 3)
# sort = np.sort(test, axis=1)
# sortflipped = np.flip(sort, axis=1)
# print(test)
# print(sort)
# print(sortflipped)

test = np.random.rand(5,3)
thresh = np.random.rand(5,1)
print(test)
print(thresh)
mask = test>thresh
masked = test*mask
print(mask)
test[:, 1]

[[0.3577727  0.39081675 0.98049294]
 [0.68183892 0.4812064  0.67377119]
 [0.66811415 0.36795265 0.57007978]
 [0.03092792 0.38152779 0.62178179]
 [0.69190498 0.23254899 0.42479153]]
[[0.44221907]
 [0.17981336]
 [0.24669232]
 [0.82597427]
 [0.18928009]]
[[False False  True]
 [ True  True  True]
 [ True  True  True]
 [False False False]
 [ True  True  True]]


array([0.39081675, 0.4812064 , 0.36795265, 0.38152779, 0.23254899])

In [10]:
# Ranking frequencies based on absolute value of real part (not updated to include 1D FFT in both directions)

phi = np.load("Phi_201snap.npy")
df = pd.DataFrame()

# Define values for k
k_values = [1, 2, 5, 10, 20, 50, 75, 100]

for k in k_values:
    sum_of_squares1D, sum_of_squares2D = 0, 0
    max1D, min1D, max2D, min2D = -1000, 1000, -1000, 1000
    
    for i in range(201):
        data = phi[:,:,i]

        # Compute 1D (in r) and 2D FFT's
        fft = np.fft.fft(data, axis=0)
        fft2 = np.fft.fft2(data)

        # 1D
        # Sort fft by absolute value of real part in descending order
        fft_sorted = np.sort(np.abs(np.real(fft.reshape(-1))))[::-1]
        
        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold = fft_sorted[k]
        mask = np.abs(np.real(fft))>threshold
        masked = fft * mask

        # Reconstruct the data from masked using inverse FFT
        newdata = np.fft.ifft(masked).real
        
        # Add squared differences to total sum, update min/max values for later
        diff1D = data - newdata
        sum_of_squares1D += np.sum(diff1D**2)
        if np.max(diff1D) > max1D:
            max1D = np.max(diff1D)
        if np.min(diff1D) < min1D:
            min1D = np.min(diff1D)

        # 2D
        # Sort fft by absolute value of real part in descending order
        fft2_sorted = np.sort(np.abs(np.real(fft2.reshape(-1))))[::-1]

        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold2 = fft2_sorted[k]
        mask2 = np.abs(np.real(fft2))>threshold2
        masked2 = fft2 * mask2
        
        # Reconstruct the data from masked2 using inverse FFT2
        newdata2 = np.fft.ifft2(masked2).real
        
        # Add squared differences to total sum, update min/max values for later
        diff2D = data - newdata2
        sum_of_squares2D += np.sum(diff2D**2)
        if np.max(diff2D) > max2D:
            max2D = np.max(diff2D)
        if np.min(diff2D) < min2D:
            min2D = np.min(diff2D)

    # Compute RMSE's and CR's of 1D and 2D cases, add to dataframe
    range1D = max1D - min1D
    rms1D = np.sqrt(sum_of_squares1D / (101*227*201))
    rmse1D = rms1D / (100*range1D)
    cr1D = 101 / k
    cr_string1D = "CR1D_k=" + str(k)
    rmse_string1D = "RMSE1D_k=" + str(k)
    df.at[0, cr_string1D] = cr1D
    df.at[0, rmse_string1D] = rmse1D

    range2D = max2D - min2D
    rms2D = np.sqrt(sum_of_squares2D / (101*227*201))
    rmse2D = rms2D / (100*range2D)
    cr2D = (101*227) / k
    cr_string2D = "CR2D_k=" + str(k)
    rmse_string2D = "RMSE2D_k=" + str(k)
    df.at[0, cr_string2D] = cr2D
    df.at[0, rmse_string2D] = rmse2D
        
df.to_pickle("analysis2.4.pkl")
df

,CR1D_k=1,RMSE1D_k=1,CR2D_k=1,RMSE2D_k=1,CR1D_k=2,RMSE1D_k=2,CR2D_k=2,RMSE2D_k=2,CR1D_k=5,RMSE1D_k=5,...,CR2D_k=50,RMSE2D_k=50,CR1D_k=75,RMSE1D_k=75,CR2D_k=75,RMSE2D_k=75,CR1D_k=100,RMSE1D_k=100,CR2D_k=100,RMSE2D_k=100
0,101.0,0.000291,22927.0,0.000262,50.5,0.000291,11463.5,0.000265,20.2,0.00028,...,458.54,0.000288,1.346667,0.000041,305.693333,0.000285,1.01,0.000034,229.27,0.000252


In [11]:
# Debugging: Plotting images for k=1 and k=2 2D FFTs

# Ranking FFT frequencies based on absolute value (sqrt(a^2+b^2))

phi = np.load("Phi_201snap.npy")
df = pd.DataFrame()
i = 160
my_dpi = 192

# Define values for k
k_values = [1, 2]
data = phi[:,:,i]

plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
plt.imshow(data)
plt.axis("off")
plt.savefig('2.4.png', bbox_inches='tight', pad_inches=0)
plt.close()

# Compute 1D (in r) and 2D FFT's


for k in k_values:
    
    # 2D
    # Sort by absolute value in descending order
    fft2 = np.fft.fft2(data)
    fft2_sorted = np.sort(np.abs(fft2.reshape(-1)))[::-1]

    # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
    threshold2 = fft2_sorted[k]
    mask2 = np.abs(fft2)>threshold2
    masked2 = fft2 * mask2
    
    # Reconstruct the data from masked2 using inverse FFT2
    newdata2 = np.fft.ifft2(masked2).real
    title = '2.4k=' + str(k) + '.png'

    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(newdata2)
    plt.axis("off")
    plt.savefig(title, bbox_inches='tight', pad_inches=0)
    plt.close()

In [57]:
# # Instead of looping through 201 images (in cell below), can do the 2D FFT on the whole phi array all at once. 
# # Easier then to do the 201*N RMSE thing.

# phi = np.load("Phi_201snap.npy")
# df = pd.DataFrame(index=np.arange(201))

# # Define values for k
# k_values = [1, 2, 5, 10, 20, 50, 75, 100]

# # Compute 1D (in r) and 2D (r, z) FFT's
# fft = np.fft.fft(phi, axis=0)
# fft2 = np.fft.fft2(phi, axes=(0, 1))

# for k in k_values:
#     # 1D
#     # Keep only the top k frequencies
#     fft_compressed1D = np.zeros_like(fft)
#     fft_compressed1D[:k] = fft[:k]
    
#     # Reconstruct the data using inverse FFT
#     reconstructed1D = np.fft.ifft(fft_compressed1D).real
    
#     # Compute the error (RMSE) and compression ratio
#     diff1D = data - reconstructed1D
#     rmse1D = np.sqrt(np.mean(diff1D**2))
#     cr1D = 101 / k
    
#     # Add to dataframe
#     # cr_string1D = "CR1D_k=" + str(k)
#     rmse_string1D = "RMSE1D_k=" + str(k)
#     # df.at[i, cr_string1D] = cr1D
#     df.at[i, rmse_string1D] = rmse1D
    
#     # 2D
#     # Keep only the top k frequencies
#     fft_compressed2D = np.zeros_like(fft2)
#     fft_compressed2D[:k, :k] = fft2[:k, :k]
    
#     # Reconstruct the data using inverse FFT
#     reconstructed2D = np.fft.ifft2(fft_compressed2D).real
    
#     # Compute the error (RMSE) and compression ratio
#     diff2D = data - reconstructed2D
#     rmse2D = np.sqrt(np.mean(diff2D**2))
#     cr2D = (101 * 227) / (k * k)
    
#     # Add to dataframe
#     # cr_string2D = "CR2D_k=" + str(k)
#     rmse_string2D = "RMSE2D_k=" + str(k)
#     # df.at[i, cr_string2D] = cr2D
#     df.at[i, rmse_string2D] = rmse2D
        
# # df.to_pickle("analysis2.4.pkl")
# df
print(fft_sorted)
fft_sorted[1,:]

[[3.91023390e-07 3.02600034e-23 2.83831527e-23 ... 7.31131276e-25
  3.19739347e-25 2.31204010e-25]
 [3.90888410e-07 1.40702981e-08 1.40702981e-08 ... 5.87196052e-14
  2.79156987e-14 2.79156987e-14]
 [3.90742210e-07 3.11546098e-08 3.11546098e-08 ... 1.77260470e-14
  1.13385706e-14 1.13385706e-14]
 ...
 [3.71843410e-08 2.19770231e-08 2.19770231e-08 ... 8.17144804e-15
  6.90696565e-15 6.90696565e-15]
 [1.94760661e-08 1.13773133e-08 1.13773133e-08 ... 1.32380756e-14
  8.24813425e-15 8.24813424e-15]
 [9.04918875e-13 9.04918875e-13 1.87787285e-13 ... 6.95332243e-16
  5.14522237e-16 5.14522237e-16]]


array([3.90888410e-07, 1.40702981e-08, 1.40702981e-08, 5.67368426e-10,
       5.67368426e-10, 2.84701509e-10, 2.84701509e-10, 1.18109339e-10,
       1.18109339e-10, 8.18027553e-11, 8.18027553e-11, 4.97951435e-11,
       4.97951435e-11, 3.88333709e-11, 3.88333709e-11, 3.78717895e-11,
       3.78717895e-11, 2.98301892e-11, 2.98301892e-11, 2.74338561e-11,
       2.74338561e-11, 2.16644274e-11, 2.16644274e-11, 1.74074583e-11,
       1.74074583e-11, 1.38852626e-11, 1.38852626e-11, 1.20638394e-11,
       1.20638394e-11, 1.17905592e-11, 1.17905592e-11, 9.57717905e-12,
       9.57717905e-12, 8.76285126e-12, 8.76285126e-12, 8.13828016e-12,
       8.13828016e-12, 7.04752033e-12, 7.04752033e-12, 6.75341804e-12,
       6.75341804e-12, 5.36544195e-12, 5.36544195e-12, 5.33818430e-12,
       5.33818430e-12, 5.03308465e-12, 5.03308465e-12, 4.59805854e-12,
       4.59805854e-12, 4.36131450e-12, 4.36131450e-12, 4.17480327e-12,
       4.17480327e-12, 3.64264376e-12, 3.64264376e-12, 3.30573204e-12,
      